In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime

Load data

In [4]:
features = pd.read_csv("temps.csv")

In [5]:
features.head()

,year,month,day,week,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend
0,2016,1,1,Fri,45,45,45.6,45,43,50,44,29
1,2016,1,2,Sat,44,45,45.7,44,41,50,44,61
2,2016,1,3,Sun,45,44,45.8,41,43,46,47,56
3,2016,1,4,Mon,44,41,45.9,40,44,48,46,53
4,2016,1,5,Tues,41,40,46.0,44,46,46,46,41


In [6]:
features.tail()

,year,month,day,week,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend
343,2016,12,27,Tues,42,42,45.2,47,41,50,47,47
344,2016,12,28,Wed,42,47,45.3,48,41,49,44,58
345,2016,12,29,Thurs,47,48,45.3,48,43,50,45,65
346,2016,12,30,Fri,48,48,45.4,57,44,46,44,42
347,2016,12,31,Sat,48,57,45.5,40,42,48,47,57


In [8]:
features.shape

(348, 12)

In [9]:
features.describe()

,year,month,day,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend
count,348.0,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000
mean,2016.0,6.477011,15.514368,62.652299,62.701149,59.760632,62.543103,57.238506,62.373563,59.772989,60.034483
std,0.0,3.498380,8.772982,12.165398,12.120542,10.527306,11.794146,10.605746,10.549381,10.705256,15.626179
min,2016.0,1.000000,1.000000,35.000000,35.000000,45.100000,35.000000,41.000000,46.000000,44.000000,28.000000
25%,2016.0,3.000000,8.000000,54.000000,54.000000,49.975000,54.000000,48.000000,53.000000,50.000000,47.750000
50%,2016.0,6.000000,15.000000,62.500000,62.500000,58.200000,62.500000,56.000000,61.000000,58.000000,60.000000
75%,2016.0,10.000000,23.000000,71.000000,71.000000,69.025000,71.000000,66.000000,72.000000,69.000000,71.000000
max,2016.0,12.000000,31.000000,117.000000,117.000000,77.400000,92.000000,77.000000,82.000000,79.000000,95.000000


In [36]:
features = pd.get_dummies(features)

In [38]:
#sepearte variable and target variable
label = features['actual']
features = features.drop('actual', axis=1)

In [39]:
feature_list = list(features.columns)

In [40]:
labels = np.array(label)
features = np.array(features)

In [41]:
from sklearn.model_selection import train_test_split
train_feature, test_feature, train_label, test_label = train_test_split(features, labels, test_size=0.25, random_state=42)

In [44]:
print train_feature.shape
print train_label.shape
print test_feature.shape
print test_label.shape

(261L, 17L)
(261L,)
(87L, 17L)
(87L,)


In [48]:
baseline_pred = test_feature[:, feature_list.index('average')]
baseline_error = abs(baseline_pred - test_label)
print round(np.mean(baseline_error), 2)

5.06


In [49]:
from sklearn.ensemble import RandomForestRegressor

#for 1000 decision tress 
rf = RandomForestRegressor(n_estimators=1000, random_state=42)
rf.fit(train_feature, train_label)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [50]:
predictions = rf.predict(test_feature)

errors = abs(predictions-test_label)
print round(np.mean(errors), 2)

3.87


In [57]:
#Mean ablsolute percentage error
mape = (errors/test_label)*100

accuracy = 100 - np.mean(mape)
print ("percentage accuracy is %s" %(accuracy))

percentage accuracy is 93.9325320799977


In [64]:
from sklearn.tree import export_graphviz
import pydot

tree = rf.estimators_[2]

#image to a dot file
export_graphviz(tree, out_file='tree.dot', feature_names=feature_list, rounded=True, precision=1)

#create graph from dot file
(graph, ) = pydot.graph_from_dot_file('tree.dot')

#to png file
graph.write_png('tree2.png')

In [66]:
importances = list(rf.feature_importances_)

feature_importances = [(feature, round(importance,2)) for feature, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key=lambda x:x[1], reverse=True)

print feature_importances

[('temp_1', 0.66), ('average', 0.15), ('forecast_noaa', 0.05), ('forecast_acc', 0.03), ('day', 0.02), ('temp_2', 0.02), ('forecast_under', 0.02), ('friend', 0.02), ('month', 0.01), ('year', 0.0), ('week_Fri', 0.0), ('week_Mon', 0.0), ('week_Sat', 0.0), ('week_Sun', 0.0), ('week_Thurs', 0.0), ('week_Tues', 0.0), ('week_Wed', 0.0)]


In [71]:
rf_most_important = RandomForestRegressor(n_estimators=1000, random_state=42)

important_indices = [feature_list.index('temp_1'), feature_list.index('average')]
#print important_indices
train_important = train_feature[:, important_indices]
test_important = test_feature[:, important_indices]

#fit into RandomForest
rf_most_important.fit(train_important, train_label)

#predictions
prediction_important = rf_most_important.predict(test_important)

#Calculate Errors and all
erros = abs(prediction_important-test_label)
print("ERROR.......", round(np.mean(erros), 2))
accuracy_important = (erros/test_label)*100
print("ACCURACY......", 100-np.mean(accuracy_important))


('ERROR.......', 3.92)
('ACCURACY......', 93.75689140426533)
